In [1]:
from pathlib import Path
import tifffile
import os
import numpy as np
from tqdm import tqdm
from stardist.models import StarDist3D
from utils_stardist import get_gpu_details, list_images, read_image, extract_nuclei_stack, segment_nuclei_3d, simulate_cytoplasm

get_gpu_details()

Device name: /device:GPU:0
Device type: GPU
GPU model: device: 0, name: NVIDIA GeForce RTX 4090 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [2]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Construct ROI path from directory_path above
roi_path = directory_path / "ROIs"

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

<h3>Define your nuclei channel and your nuclear segmentation parameters</h3>

Modify the values for <code>slicing factor</code>, <code>nuclei_channel</code>, <code>segmentation_type</code> and your Stardist <code>model_name</code>


In [3]:
# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

n_tiles=(6,6,3)

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "3D"

# Nuclear segmentation model type ("Stardist")
# Choose your Stardist fine-tuned model (model_name) from stardist_models folder
model_name = "MEC0.1"

# Model loading 
model = StarDist3D(None, name=model_name, basedir='stardist_models')

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.583933, nms_thresh=0.3.


<h3>Predict nuclei labels and store them as .tiff files</h3>

In [4]:
# List of subfolder names
try: 
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]
    print(f"The following regions of interest will be analyzed: {roi_names}")

except FileNotFoundError:
    roi_names = ["full_image"]

for image in tqdm(images):

    # Generate maximum intensity projection and extract filename
    img, filename = read_image (images[0], slicing_factor=None)

    # Slice the nuclei stack
    nuclei_img = extract_nuclei_stack(img, nuclei_channel)
    
    for roi_name in roi_names:
    
        print(f"Generating nuclei predictions for {roi_name} ROI")

        # Construct path to read ROI
        roi_path = directory_path / "ROIs" / roi_name / f"{filename}.tiff"

        try:

            # Read the .tiff files containing the user-defined ROIs
            roi = tifffile.imread(roi_path)

            # We will create a mask where roi is greater than or equal to 1
            mask = (roi >= 1).astype(np.uint8)

            # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
            masked_nuclei_img = np.where(mask, nuclei_img, 0)

        except FileNotFoundError:
            # If no ROI is saved the script will predict nuclei in the entire nuclei_img input
            masked_nuclei_img = nuclei_img

        # Segment nuclei and return labels
        nuclei_labels = segment_nuclei_3d(nuclei_img, model, n_tiles)

        # Save nuclei labels as .tiff files to reuse them later
        # Create nuclei_predictions directory if it does not exist
        try:
            os.makedirs(directory_path / "nuclei_preds" / segmentation_type / roi_name)
        except FileExistsError:
            pass

        # Construct path to store
        nuclei_preds_path = directory_path / "nuclei_preds" / segmentation_type/ roi_name / f"{filename}.tiff"

        # Save nuclei labels as .tiff
        tifffile.imwrite(nuclei_preds_path, nuclei_labels)

  0%|          | 0/2 [00:00<?, ?it/s]

Image analyzed: HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 14, 3803, 2891)
Compressed Array shape: (4, 14, 3803, 2891)
Generating nuclei predictions for full_image ROI


 50%|█████     | 1/2 [03:21<03:21, 201.43s/it]

Image analyzed: HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 14, 3803, 2891)
Compressed Array shape: (4, 14, 3803, 2891)
Generating nuclei predictions for full_image ROI


100%|██████████| 2/2 [06:41<00:00, 200.71s/it]
